<a href="https://colab.research.google.com/github/karengarm/Project_CS330/blob/main/ComputeTaskEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#@title Installations
!pip install transformers --quiet
!pip install pytorch-transformers --quiet
!pip install datasets --quiet
!wget -nc 'https://raw.githubusercontent.com/bymarkone/pytorch-transformers/master/examples/utils_squad.py' --quiet
!wget -nc 'https://raw.githubusercontent.com/bymarkone/pytorch-transformers/master/examples/utils_qa.py' --quiet
!wget -nc 'https://raw.githubusercontent.com/bymarkone/pytorch-transformers/master/examples/utils_squad_evaluate.py' --quiet
!git clone https://github.com/tuvuumass/task-transferability.git

#use transformers inside task-transferability with a different name
!mv task-transferability/transformers task-transferability/transformersx
!rm task-transferability/utils_squad.py
!rm task-transferability/utils_squad_evaluate.py
import sys
sys.path.insert(0, 'task-transferability')



     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 7.6 MB 82.8 MB/s 
     |████████████████████████████████| 182 kB 97.8 MB/s 
     |████████████████████████████████| 176 kB 4.8 MB/s 
     |████████████████████████████████| 1.3 MB 60.4 MB/s 
     |████████████████████████████████| 880 kB 70.2 MB/s 
     |████████████████████████████████| 132 kB 83.4 MB/s 
     |████████████████████████████████| 10.2 MB 83.7 MB/s 
     |████████████████████████████████| 79 kB 9.9 MB/s 
     |████████████████████████████████| 140 kB 89.6 MB/s 
     |████████████████████████████████| 127 kB 94.7 MB/s 
     |████████████████████████████████| 451 kB 4.6 MB/s 
     |████████████████████████████████| 132 kB 76.3 MB/s 
     |████████████████████████████████| 212 kB 98.4 MB/s 
Cloning into 'task-transferability'...
remote: Enumerating objects: 118, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 11

In [ ]:
#@title Datasets download and Preparation


# squad_v2
!wget -nc 'https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json' --quiet -P datasets/squad_v2

# squad_v1
!wget -nc 'https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json' --quiet -P datasets/squad_v1

# drop
!rm -rf datasets/tmp
!rm -rf datasets/drop

!mkdir -p datasets/drop

!wget -nc 'https://ai2-public-datasets.s3.amazonaws.com/drop/drop_dataset.zip' --quiet -P datasets/tmp
!unzip -qq datasets/tmp/drop_dataset.zip -d datasets/tmp

import json

with open('datasets/tmp/drop_dataset/drop_dataset_train.json', 'r') as infile, open('datasets/drop/train.json', 'w') as outfile:
  drop = json.load(infile)
  data = { "data": [] }
  for key in drop:
    paragraphs = {
        "paragraphs": []
    }
    paragraph = {
        "context": drop[key]["passage"],
        "qas": []
    }
    for qa_pair in drop[key]["qa_pairs"]:
      qa = {
          "answers": [  ],
          "id": qa_pair["query_id"],
          "is_impossible": False,
          "question": qa_pair["question"]
      }
      answer = {
          "answer_start": 0,
          "text": qa_pair["answer"]["number"] + " ".join(qa_pair["answer"]["spans"])
      }
      qa["answers"].append(answer)
      paragraph["qas"].append(qa)

    paragraphs["paragraphs"].append(paragraph)
    data["data"].append(paragraphs)

  json.dump(data, outfile)

# sst-2
!wget -nc 'https://raw.githubusercontent.com/awake020/SST-2-sentence-dataset/master/train.tsv' --quiet -P datasets/sst2
import csv
with open('datasets/sst2/train.tsv', 'r') as infile, open('datasets/sst2/train2.tsv', 'a') as outfile:
    # output dict needs a list for new column ordering
    fieldnames = ['sentence', 'label']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames, delimiter='\t')
    # reorder the header first
    writer.writeheader()
    for row in csv.DictReader(infile, delimiter='\t'):
        # writes the reordered rows to the new file
        writer.writerow(row)
!rm datasets/sst2/train.tsv
!mv datasets/sst2/train2.tsv datasets/sst2/train.tsv

# duorc

!wget -nc 'https://raw.githubusercontent.com/duorc/duorc/master/dataset/ParaphraseRC_train.json' -P datasets/tmp
!wget -nc 'https://raw.githubusercontent.com/duorc/duorc/master/dataset/SelfRC_train.json' --quiet -P datasets/tmp
!mkdir -p datasets/duorc_p
!mkdir -p datasets/duorc_r

import json

def adapt_duorc(infilename, outfilename):
  with open(infilename, 'r') as infile, open(outfilename, 'w') as outfile:
    duo = json.load(infile)
    data = { "data": [] }
    for item in duo:
      paragraphs = {
          "paragraphs": []
      }
      paragraph = {
          "context": item["plot"],
          "qas": []
      }
      for qa_pair in item["qa"]:
        qa = {
            "answers": [],
            "id": qa_pair["id"],
            "is_impossible": qa_pair["id"],
            "question": qa_pair["question"]
        }
        for qa_answer in qa_pair["answers"]:
          answer = {
              "answer_start": 0,
              "text": qa_answer
          }
          qa["answers"].append(answer)
        
        paragraph["qas"].append(qa)

      paragraphs["paragraphs"].append(paragraph)
      data["data"].append(paragraphs)

    json.dump(data, outfile)

adapt_duorc('datasets/tmp/ParaphraseRC_train.json', 'datasets/duorc_p/train.json')
adapt_duorc('datasets/tmp/SelfRC_train.json', 'datasets/duorc_r/train.json')

In [ ]:
#@title Imports

""" Compute TaskEmb for question answering tasks."""

from __future__ import absolute_import, division, print_function

import argparse
import logging
import os
import random
import glob
import timeit
import json


import numpy as np
import torch
from torch.distributions.normal import Normal
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset, Subset) 
from torch.utils.data.distributed import DistributedSampler

try:
    from torch.utils.tensorboard import SummaryWriter
except:
    from tensorboardX import SummaryWriter

from tqdm import tqdm, trange

from transformersx import (WEIGHTS_NAME,
                          BertConfig_TaskEmbeddings as BertConfig,
                          BertForQuestionAnswering_TaskEmbeddings as BertForQuestionAnswering,
                          BertForSequenceClassification_TaskEmbeddings as BertForSequenceClassification,
                          BertTokenizer)

from transformersx import glue_output_modes as output_modes
from transformersx import glue_processors as processors
from transformersx import glue_convert_examples_to_features as convert_examples_to_features_glue


from transformers import AdamW, get_linear_schedule_with_warmup

from datasets import load_dataset

from utils_squad import read_squad_examples
from utils_squad import convert_examples_to_features as convert_examples_to_features_squad

from utils_qa import read_qa_examples
from utils_qa import convert_examples_to_features as convert_examples_to_features_qa

# The follwing import is the official SQuAD evaluation script (2.0).
# You can remove it from the dependencies if you are using this script outside of the library
# We've added it here for automated tests (see examples/test_examples.py file)
from utils_squad_evaluate import EVAL_OPTS, main as evaluate_on_squad

logger = logging.getLogger(__name__)

MODEL_CLASSES_QA = {
    'bert': (BertConfig, BertForQuestionAnswering, BertTokenizer)
}

MODEL_CLASSES_CR = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer)
}



In [ ]:
#@title Load Data
def load_examples(dataset_name):
    dataset = load_dataset(dataset_name, split='train')
    print(dataset)
    return dataset

def load_and_cache_examples_QA(args, tokenizer, evaluate=False, output_examples=False):
    # Load data features from cache or dataset file
    input_file = args.predict_file if evaluate else args.train_file
    cached_features_file = os.path.join(os.path.dirname(input_file), 'cached_{}_{}_{}'.format(
        'dev' if evaluate else 'train',
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(args.max_seq_length)))
    if os.path.exists(cached_features_file) and not args.overwrite_cache and not output_examples:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", input_file)
        
        if "Squad" in args.dataset_name: 
            examples = read_squad_examples(input_file=input_file,
                                                is_training=not evaluate,
                                                version_2_with_negative=args.version_2_with_negative)
            features = convert_examples_to_features_squad(examples=examples,
                                                tokenizer=tokenizer,
                                                max_seq_length=args.max_seq_length,
                                                doc_stride=args.doc_stride,
                                                max_query_length=args.max_query_length,
                                                is_training=not evaluate,
                                                cls_token_segment_id=0,
                                                pad_token_segment_id=0,
                                                cls_token_at_end=False)
            
        else:
            examples = read_qa_examples(input_file=input_file,
                                                is_training=not evaluate,
                                                version_2_with_negative=args.version_2_with_negative)
            features = convert_examples_to_features_qa(examples=examples,
                                                tokenizer=tokenizer,
                                                max_seq_length=args.max_seq_length,
                                                doc_stride=args.doc_stride,
                                                max_query_length=args.max_query_length,
                                                is_training=not evaluate,
                                                cls_token_segment_id=0,
                                                pad_token_segment_id=0,
                                                cls_token_at_end=False)


        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
    all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)
    if evaluate:
        all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_example_index, all_cls_index, all_p_mask)
    else:
        all_start_positions = torch.tensor([f.start_position for f in features], dtype=torch.long)
        all_end_positions = torch.tensor([f.end_position for f in features], dtype=torch.long)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids,
                                all_start_positions, all_end_positions,
                                all_cls_index, all_p_mask)

    if output_examples:
        return dataset, examples, features
    return dataset


def load_and_cache_examples_CR(args, task, tokenizer, evaluate=False):
    processor = processors[task]()
    output_mode = output_modes[task]
    # Load data features from cache or dataset file
    cached_features_file = os.path.join(args.data_dir, 'cached_{}_{}_{}_{}'.format(
        'dev' if evaluate else 'train',
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(args.max_seq_length),
        str(task)))
    if os.path.exists(cached_features_file) and not args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        label_list = processor.get_labels()
        examples = processor.get_dev_examples(args.data_dir) if evaluate else processor.get_train_examples(
            args.data_dir)
        features = convert_examples_to_features_glue(examples,
                                                tokenizer,
                                                label_list=label_list,
                                                max_length=args.max_seq_length,
                                                output_mode=output_mode,
                                                pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                                pad_token_segment_id=0,
                                                )

        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    return dataset

In [ ]:
#@title Calculate Embeddings QA

def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def to_list(tensor):
    return tensor.detach().cpu().tolist()


def get_model_outputs(model, key):
        if key == 'multihead_output':
            # get list (layers) of multihead module outputs
            return [layer.attention.self.multihead_output for layer in model.encoder.layer]
        elif key == 'layer_output':
            # get list of encoder LayerNorm layer outputs
            return [layer.output.layer_output for layer in model.encoder.layer]
        elif key == 'cls_output':
            # get the final output of the model
            return model.pooler.cls_output
        else:
            raise ValueError("Key not found: %s" % (key))


def compute_Fisher_QA(args, model, input_mask, total_tokens):
    outputs = {}

    base_model = model.bert
    for name, parameter in base_model.named_parameters():
        if parameter.requires_grad:
            score = parameter.grad if args.feature_type == 'grads' else parameter
            if score is not None and name not in outputs:
                score = score ** args.pow
                outputs[name] = score
    # # activations
    for key in ['multihead_output', 'layer_output']:
        model_outputs = get_model_outputs(base_model, key=key)
        for i in range(base_model.config.num_hidden_layers):
            name = 'encoder.layer.{}.{}'.format(i, key)
            model_outputs_i = model_outputs[i].grad if args.feature_type == 'grads' else model_outputs[i]

            if model_outputs_i is not None:
                score = torch.einsum("ijk,ij->ijk", [model_outputs_i,   # batch_size x max_seq_length x hidden_size
                                                 input_mask.float()])   # batch_size x max_seq_length
                if score is not None and name not in outputs:
                    score = score.sum(0).sum(0)
                    score = score ** args.pow
                    # normalize
                    score = score / total_tokens
                    outputs[name] = score
    # cls output
    name = 'cls_output'
    score = get_model_outputs(base_model, key=name).grad if args.feature_type == 'grads' else get_model_outputs(base_model, key=name) # batch_size x hidden_size

    if score is not None and name not in outputs:
        score = score.sum(0)
        score = score ** args.pow
        # normalize
        score = score / total_tokens
        outputs[name] = score

    # task-specific layer
    for name, parameter in model.named_parameters():
        if args.model_type not in name:
            score = parameter.grad if args.feature_type == 'grads' else parameter
            if score is not None and name not in outputs:
                score = score ** args.pow
                outputs[name] = score

    return outputs


def compute_Fisher_no_labels_QA(args, model, input_mask, start_logits, end_logits):
    total_tokens = input_mask.float().detach().sum().data

    if args.num_softmax_classifiers > 1:
        raise ValueError("Not implemented.")
    else:
        # start_logits: batch_size x max_seq_length
        # end_logits: batch_size x max_seq_length
        start_softmax_logits = torch.softmax(start_logits, dim=1)
        end_softmax_logits = torch.softmax(end_logits, dim=1)
        sampled_start_indices = torch.multinomial(start_softmax_logits, args.num_trials_for_FIM, True)
        sampled_end_indices = torch.multinomial(end_softmax_logits, args.num_trials_for_FIM, True)
        log_start_softmax_logits = torch.log(start_softmax_logits)
        log_end_softmax_logits = torch.log(end_softmax_logits)

        sampled_log_start_softmax_logits = torch.gather(log_start_softmax_logits, dim=1, index=sampled_start_indices)
        sampled_log_start_softmax_logits = \
            sampled_log_start_softmax_logits.sum(0).sum(0) / sampled_log_start_softmax_logits.numel()

        sampled_log_end_softmax_logits = torch.gather(log_end_softmax_logits, dim=1, index=sampled_end_indices)
        sampled_log_end_softmax_logits = \
            sampled_log_end_softmax_logits.sum(0).sum(0) / sampled_log_end_softmax_logits.numel()

        sampled_log_softmax_logits = (sampled_log_start_softmax_logits + sampled_log_end_softmax_logits) / 2.0

        model.zero_grad()
        if args.finetune_classifier:
            sampled_log_softmax_logits.backward(retain_graph=True)
        else:
            sampled_log_softmax_logits.backward()
        outputs = compute_Fisher_QA(args, model, input_mask, total_tokens)
    return outputs


def compute_Fisher_with_labels_QA(args, model, input_mask, loss):
    total_tokens = input_mask.float().detach().sum().data

    model.zero_grad()
    loss.backward()
    outputs = compute_Fisher_QA(args, model, input_mask, total_tokens)
    return outputs


def compute_taskemb_QA(args, train_dataset, model):
    """ Feed task data through the model """
    tb_writer = SummaryWriter()

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.batch_size)

    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_epochs

    if args.finetune_feature_extractor and not args.finetune_classifier:
        raise ValueError("finetune_classifier should be True when finetune_feature_extractor is True.")

    if args.finetune_classifier:
        model.train()

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']

        if args.finetune_feature_extractor:
            optimizer_grouped_parameters = [
                {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                 'weight_decay': args.weight_decay},
                {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                 'weight_decay': 0.0}
            ]
        else:
            optimizer_grouped_parameters = [
                {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)
                            and args.model_type not in n],
                 'weight_decay': args.weight_decay},
                {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)
                            and args.model_type not in n],
                 'weight_decay': 0.0}
            ]

        optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps,
                                                    num_training_steps=t_total)

    else:
        model.eval()
        optimizer = None
        scheduler = None

    logger.info("***** Compute TaskEmb *****")
    logger.info("Num examples = %d", len(train_dataset))
    logger.info("Batch size = %d", args.batch_size)

    total_num_examples = 0
    model.zero_grad()
    train_iterator = trange(int(args.num_epochs), desc="Epoch", disable=False)
    set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
    global_feature_dict = {}
    for _ in train_iterator:
        num_examples = 0
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {'input_ids':       batch[0],
                      'attention_mask':  batch[1],
                      'token_type_ids': batch[2],
                      'start_positions': batch[3],
                      'end_positions':   batch[4]}

            outputs = model(**inputs)

            loss, start_logits, end_logits = outputs[0], outputs[1], outputs[2]

            input_mask = inputs['attention_mask']

            if not args.use_labels:
                feature_dict = compute_Fisher_no_labels_QA(args, model, input_mask, start_logits, end_logits)
            else:
                feature_dict = compute_Fisher_with_labels_QA(args, model, input_mask, loss)
            ###
            if len(global_feature_dict) == 0:
                for key in feature_dict:
                    global_feature_dict[key] = feature_dict[key].detach().cpu().numpy()
            else:
                for key in feature_dict:
                    global_feature_dict[key] += feature_dict[key].detach().cpu().numpy()

            if ((not args.use_labels) and args.finetune_classifier):
                model.zero_grad()
                loss.backward()

            if args.finetune_classifier:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                scheduler.step()  # Update learning rate schedule
                optimizer.step()

            model.zero_grad()
            num_examples += inputs['input_ids'].size(0)
        total_num_examples += num_examples

    # Normalize
    for key in global_feature_dict:
        global_feature_dict[key] = global_feature_dict[key] / total_num_examples

    # Save features
    for key in global_feature_dict:
        np.save(os.path.join(args.output_dir, '{}.npy'.format(key)), global_feature_dict[key])


def main_QA(args):
    # ## Other parameters
    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and not args.overwrite_output_dir:
        raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))

    # Create output directory if needed
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    with open(os.path.join(args.output_dir, 'run_args.txt'), 'w') as f:
        f.write(json.dumps(args.__dict__, indent=2))
        f.close()

    # Setup CUDA, GPU training
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    if args.n_gpu > 1:
        raise ValueError("This code only supports a single GPU.")

    # Setup logging
    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
    logger.warning("Device: %s, n_gpu: %s", device, args.n_gpu)

    # Set seed
    set_seed(args)

    args.model_type = args.model_type.lower()
    config_class, model_class, tokenizer_class = MODEL_CLASSES_QA[args.model_type]
    config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                          num_softmax_classifiers=args.num_softmax_classifiers,
                                          retain_gradients=args.retain_gradients,
                                          do_pooling=args.do_pooling,
                                          cache_dir=args.cache_dir if args.cache_dir else None)
    tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
                                                do_lower_case=args.do_lower_case,
                                                cache_dir=args.cache_dir if args.cache_dir else None)
    model = model_class.from_pretrained(args.model_name_or_path,
                                        from_tf=bool('.ckpt' in args.model_name_or_path),
                                        config=config,
                                        cache_dir=args.cache_dir if args.cache_dir else None)

    model.to(args.device)

    logger.info("List of model named parameters:")
    for n, p in list(model.named_parameters()):
        logger.info("%s", n)
    logger.info("Training/evaluation parameters %s", args)
    # Good practice: save your training arguments together with the trained model
    torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))
    tokenizer.save_pretrained(args.output_dir)

    train_dataset = load_and_cache_examples_QA(args, tokenizer, evaluate=False, output_examples=False)

    # train_dataset = load_examples(args.dataset_name)

    if args.data_subset > 0:
        train_dataset = Subset(train_dataset, list(range(min(args.data_subset, len(train_dataset)))))

    compute_taskemb_QA(args, train_dataset, model)





In [ ]:
#@title Calculate Embeddings CR

def compute_Fisher_CR(args, model, input_mask, total_tokens):
    outputs = {}

    base_model = model.bert
    for name, parameter in base_model.named_parameters():
        if parameter.requires_grad:
            score = parameter.grad if args.feature_type == 'grads' else parameter
            if score is not None and name not in outputs:
                score = score ** args.pow
                outputs[name] = score
    # activations
    for key in ['multihead_output', 'layer_output']:
        model_outputs = base_model._get_model_outputs(key=key)
        for i in range(base_model.config.num_hidden_layers):
            name = 'encoder.layer.{}.{}'.format(i, key)
            model_outputs_i = model_outputs[i].grad if args.feature_type == 'grads' else model_outputs[i]

            if model_outputs_i is not None:
                score = torch.einsum("ijk,ij->ijk", [model_outputs_i,   # batch_size x max_seq_length x hidden_size
                                                 input_mask.float()])   # batch_size x max_seq_length
                if score is not None and name not in outputs:
                    score = score.sum(0).sum(0)
                    score = score ** args.pow
                    # normalize
                    score = score / total_tokens
                    outputs[name] = score
    # cls output
    name = 'cls_output'
    score = base_model._get_model_outputs(key=name).grad if args.feature_type == 'grads' else base_model._get_model_outputs(key=name) # batch_size x hidden_size

    if score is not None and name not in outputs:
        score = score.sum(0)
        score = score ** args.pow
        # normalize
        score = score / total_tokens
        outputs[name] = score

    # task-specific layer
    for name, parameter in model.named_parameters():
        if args.model_type not in name:
            score = parameter.grad if args.feature_type == 'grads' else parameter
            if score is not None and name not in outputs:
                score = score ** args.pow
                outputs[name] = score

    return outputs


def compute_Fisher_no_labels_CR(args, model, input_mask, logits):
    total_tokens = input_mask.float().detach().sum().data

    if args.num_labels == 1:
        #  We are doing regression
        if args.num_softmax_classifiers > 1:
            raise ValueError("Not implemented.")
        else:
            normal = Normal(logits, scale=torch.tensor([args.FIM_scale]).to(logits.device))
            num_trials = 0
            sampled_logits = None
            while num_trials < args.num_trials_for_FIM:
                sample = normal.sample(sample_shape=(1,)).squeeze(0)
                sample.to(logits.device)
                if ((sample >= 0.0) & (sample <= 5.0)).sum().item() == logits.size(0):
                    num_trials += 1
                    if sampled_logits is None:
                        sampled_logits = sample
                    else:
                        sampled_logits = torch.cat((sampled_logits, sample), dim=1)
            sampled_logits = -((sampled_logits - logits) ** 2) / (2 * (args.FIM_scale ** 2))
            sampled_logits = sampled_logits.sum(0).sum(0) / sampled_logits.numel()

            model.zero_grad()
            if args.finetune_classifier:
                sampled_logits.backward(retain_graph=True)
            else:
                sampled_logits.backward()
            outputs = compute_Fisher_CR(args, model, input_mask, total_tokens)
    else:
        #  We are doing classification
        if args.num_softmax_classifiers > 1:
            raise ValueError("Not implemented.")
        else:
            softmax_logits = torch.softmax(logits, dim=1) # batch_size x num_labels
            sampled_indices = torch.multinomial(softmax_logits, args.num_trials_for_FIM, True)
            log_softmax_logits = torch.log(softmax_logits)
            sampled_log_softmax_logits = torch.gather(log_softmax_logits, dim=1, index=sampled_indices)

            sampled_log_softmax_logits = sampled_log_softmax_logits.sum(0).sum(0) / sampled_log_softmax_logits.numel()

            model.zero_grad()
            if args.finetune_classifier:
                sampled_log_softmax_logits.backward(retain_graph=True)
            else:
                sampled_log_softmax_logits.backward()
            outputs = compute_Fisher_CR(args, model, input_mask, total_tokens)
    return outputs


def compute_Fisher_with_labels_CR(args, model, input_mask, loss):
    total_tokens = input_mask.float().detach().sum().data

    model.zero_grad()
    loss.backward()
    outputs = compute_Fisher_CR(args, model, input_mask, total_tokens)
    return outputs


def compute_taskemb_CR(args, train_dataset, model):
    """ Feed task data through the model """
    tb_writer = SummaryWriter()

    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.batch_size)

    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_epochs

    if args.finetune_feature_extractor and not args.finetune_classifier:
        raise ValueError("finetune_classifier should be True when finetune_feature_extractor is True.")

    if args.finetune_classifier:
        model.train()

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']

        if args.finetune_feature_extractor:
            optimizer_grouped_parameters = [
                {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                 'weight_decay': args.weight_decay},
                {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                 'weight_decay': 0.0}
            ]
        else:
            optimizer_grouped_parameters = [
                {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)
                            and args.model_type not in n],
                 'weight_decay': args.weight_decay},
                {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)
                            and args.model_type not in n],
                 'weight_decay': 0.0}
            ]

        optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps,
                                                    num_training_steps=t_total)

    else:
        model.eval()
        optimizer = None
        scheduler = None

    logger.info("***** Compute TaskEmb *****")
    logger.info("Num examples = %d", len(train_dataset))
    logger.info("Batch size = %d", args.batch_size)

    total_num_examples = 0
    model.zero_grad()
    train_iterator = trange(int(args.num_epochs), desc="Epoch", disable=False)
    set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
    global_feature_dict = {}
    for _ in train_iterator:
        num_examples = 0
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
        for step, batch in enumerate(epoch_iterator):
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {'input_ids': batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2],
                      'labels': batch[3]}

            outputs = model(**inputs)

            loss, logits = outputs[0], outputs[1]

            input_mask = inputs['attention_mask']

            if not args.use_labels:
                feature_dict = compute_Fisher_no_labels_CR(args, model, input_mask, logits)
            else:
                feature_dict = compute_Fisher_with_labels_CR(args, model, input_mask, loss)
            ###
            if len(global_feature_dict) == 0:
                for key in feature_dict:
                    global_feature_dict[key] = feature_dict[key].detach().cpu().numpy()
            else:
                for key in feature_dict:
                    global_feature_dict[key] += feature_dict[key].detach().cpu().numpy()

            if ((not args.use_labels) and args.finetune_classifier):
                model.zero_grad()
                loss.backward()

            if args.finetune_classifier:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                scheduler.step()  # Update learning rate schedule
                optimizer.step()

            model.zero_grad()
            num_examples += inputs['input_ids'].size(0)
        total_num_examples += num_examples

    # Normalize
    for key in global_feature_dict:
        global_feature_dict[key] = global_feature_dict[key] / total_num_examples

    # Save features
    for key in global_feature_dict:
        np.save(os.path.join(args.output_dir, '{}.npy'.format(key)), global_feature_dict[key])


def main_CR(args):
    if os.path.exists(args.output_dir) and os.listdir( args.output_dir) and not args.overwrite_output_dir:
        raise ValueError( "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format( args.output_dir))

    # Create output directory if needed
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    with open(os.path.join(args.output_dir, 'run_args.txt'), 'w') as f:
        f.write(json.dumps(args.__dict__, indent=2))
        f.close()

    # Setup CUDA, GPU training
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    if args.n_gpu > 1:
        raise ValueError("This code only supports a single GPU.")

    # Setup logging
    logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt='%m/%d/%Y %H:%M:%S',
                        level=logging.INFO)
    logger.warning("Device: %s, n_gpu: %s", device, args.n_gpu)

    # Set seed
    set_seed(args)

    # Prepare GLUE task
    args.task_name = args.task_name.lower()
    if args.task_name not in processors:
        raise ValueError("Task not found: %s" % (args.task_name))
    processor = processors[args.task_name]()
    args.output_mode = output_modes[args.task_name]
    label_list = processor.get_labels()
    args.num_labels = len(label_list)

    args.model_type = args.model_type.lower()
    config_class, model_class, tokenizer_class = MODEL_CLASSES_CR[args.model_type]
    config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                          num_labels=args.num_labels,
                                          finetuning_task=args.task_name,
                                          num_softmax_classifiers=args.num_softmax_classifiers,
                                          retain_gradients=args.retain_gradients,
                                          do_pooling=args.do_pooling,
                                          cache_dir=args.cache_dir if args.cache_dir else None)
    tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
                                                do_lower_case=args.do_lower_case,
                                                cache_dir=args.cache_dir if args.cache_dir else None)
    model = model_class.from_pretrained(args.model_name_or_path,
                                        from_tf=bool('.ckpt' in args.model_name_or_path),
                                        config=config,
                                        cache_dir=args.cache_dir if args.cache_dir else None)

    model.to(args.device)

    logger.info("List of model named parameters:")
    for n, p in list(model.named_parameters()):
        logger.info("%s", n)
    logger.info("Training/evaluation parameters %s", args)
    # Good practice: save your training arguments together with the trained model
    torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))
    tokenizer.save_pretrained(args.output_dir)

    train_dataset = load_and_cache_examples_CR(args, args.task_name, tokenizer, evaluate=False)
    if args.data_subset > 0:
        train_dataset = Subset(train_dataset, list(range(min(args.data_subset, len(train_dataset)))))
    compute_taskemb_CR(args, train_dataset, model)



In [ ]:
#@title Run task embeddings for QA task with Squad_v2

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    def __getstate__(self): return self.__dict__
    def __setstate__(self, d): self.__dict__.update(d)


if __name__ == '__main__':

    main_args = {
        'train_file': 'datasets/squad_v2/train-v2.0.json',
        'model_type': 'bert',
        'model_name_or_path': 'bert-base-uncased',
        'output_dir': './gdrive/MyDrive/colab-results/squad_v2',
        'dataset_name': 'squad',

        'num_softmax_classifiers': 1,           # Number of softmax classifiers on top of Bert's output.
        'pow': 2.0,                             # Return features to the power pow.
        'feature_type': '',                     # The type of the features selected in ['grads', 'weights']"
        'retain_gradients': True,               # Whether to retain gradients at each layer output of the feature extractor.
        'do_pooling': 'True',                   # Whether to pool the feature extractor.
        'use_labels': 'True',                   # Whether to use training labels or sample from the model's predictive distribution \n" "pθ(y|xn), e.g., to compute the theoretical Fisher information.
        'num_trials_for_FIM': 100,              # Number of trials to sample from the model's predictive distribution pθ(y|xn).
        'FIM_scale': 0.25,                      # Standard deviation of the distribution used to compute the theoretical FIM.
        'finetune_classifier': 'False',         # Whether to fine-tune the final classifier.
        'finetune_feature_extractor': 'False',  # Whether to fine-tune the feature extractor.

        'batch_size': 15,
        'data_subset': 60,
        'overwrite_output_dir': True,
        'device': 'cuda',
        'seed': 42,

        'config_name': '',                      # Pretrained config name or path if not the same as model_name
        'tokenizer_name': '',                   # Pretrained tokenizer name or path if not the same as model_name
        'cache_dir': './cache',                        # Where do you want to store the pre-trained models downloaded from s3
        'version_2_with_negative': True,         # If true, the SQuAD examples contain some that do not have an answer.
        'null_score_diff_threshold': 0.0,       # If null_score - best_non_null is greater than the threshold predict null.
        'max_seq_length': 384,                  # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded.
        'doc_stride': 128,                      # When splitting up a long document into chunks, how much stride to take between chunks.
        'max_query_length': 64,                 # The maximum number of tokens for the question. Questions longer than this will " "be truncated to this length.
        'do_train': True,                       # Whether to run training.
        'do_lower_case': True,                  # Set this flag if you are using an uncased model.
        'learning_rate': 5e-5,                  # The initial learning rate for Adam."
        'gradient_accumulation_steps': 1,       # Number of updates steps to accumulate before performing a backward/update pass.")
        'weight_decay': 0.0,                    # Weight deay if we apply some.
        'adam_epsilon': 1e-8,                   # Epsilon for Adam optimizer.")
        'max_grad_norm': 1.0,                   # Max gradient norm."
        'num_epochs': 1.0,                      # Total number of training epochs to perform.
        'warmup_steps': 0,                      # Linear warmup over warmup_steps.
        'n_best_size': 20,                      # The total number of n-best predictions to generate in the nbest_predictions.json output file.
        'max_answer_length': 30,                # The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        'verbose_logging': True,                # If true, all of the warnings related to data processing will be printed. " "A number of warnings are expected for a normal SQuAD evaluation.
        'logging_steps': 50,                    # Log every X updates steps.
        'no_cuda': False,                       # Whether not to use CUDA when available
        'overwrite_cache': True,               # Overwrite the cached training and evaluation sets")
        'save': 'all',                          # Select load mode from ['all', '0', '1', '2', '3', ...]")
    }

    main_args = dotdict(main_args)

    main_QA(main_args)


In [ ]:
#@title Run task embeddings for QA task with Squad_v1

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    def __getstate__(self): return self.__dict__
    def __setstate__(self, d): self.__dict__.update(d)


if __name__ == '__main__':

    main_args = {
        'train_file': 'datasets/squad_v1/train-v1.1.json',
        'model_type': 'bert',
        'model_name_or_path': 'bert-base-uncased',
        'output_dir': './gdrive/MyDrive/colab-results/squad_v1',
        'dataset_name': 'squad',

        'num_softmax_classifiers': 1,           # Number of softmax classifiers on top of Bert's output.
        'pow': 2.0,                             # Return features to the power pow.
        'feature_type': '',                     # The type of the features selected in ['grads', 'weights']"
        'retain_gradients': True,               # Whether to retain gradients at each layer output of the feature extractor.
        'do_pooling': 'True',                   # Whether to pool the feature extractor.
        'use_labels': 'True',                   # Whether to use training labels or sample from the model's predictive distribution \n" "pθ(y|xn), e.g., to compute the theoretical Fisher information.
        'num_trials_for_FIM': 100,              # Number of trials to sample from the model's predictive distribution pθ(y|xn).
        'FIM_scale': 0.25,                      # Standard deviation of the distribution used to compute the theoretical FIM.
        'finetune_classifier': 'False',         # Whether to fine-tune the final classifier.
        'finetune_feature_extractor': 'False',  # Whether to fine-tune the feature extractor.

        'batch_size': 15,
        'data_subset': 60,
        'overwrite_output_dir': True,
        'device': 'cuda',
        'seed': 42,

        'config_name': '',                      # Pretrained config name or path if not the same as model_name
        'tokenizer_name': '',                   # Pretrained tokenizer name or path if not the same as model_name
        'cache_dir': './cache',                        # Where do you want to store the pre-trained models downloaded from s3
        'version_2_with_negative': False,         # If true, the SQuAD examples contain some that do not have an answer.
        'null_score_diff_threshold': 0.0,       # If null_score - best_non_null is greater than the threshold predict null.
        'max_seq_length': 384,                  # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded.
        'doc_stride': 128,                      # When splitting up a long document into chunks, how much stride to take between chunks.
        'max_query_length': 64,                 # The maximum number of tokens for the question. Questions longer than this will " "be truncated to this length.
        'do_train': True,                       # Whether to run training.
        'do_lower_case': True,                  # Set this flag if you are using an uncased model.
        'learning_rate': 5e-5,                  # The initial learning rate for Adam."
        'gradient_accumulation_steps': 1,       # Number of updates steps to accumulate before performing a backward/update pass.")
        'weight_decay': 0.0,                    # Weight deay if we apply some.
        'adam_epsilon': 1e-8,                   # Epsilon for Adam optimizer.")
        'max_grad_norm': 1.0,                   # Max gradient norm."
        'num_epochs': 1.0,                      # Total number of training epochs to perform.
        'warmup_steps': 0,                      # Linear warmup over warmup_steps.
        'n_best_size': 20,                      # The total number of n-best predictions to generate in the nbest_predictions.json output file.
        'max_answer_length': 30,                # The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        'verbose_logging': True,                # If true, all of the warnings related to data processing will be printed. " "A number of warnings are expected for a normal SQuAD evaluation.
        'logging_steps': 50,                    # Log every X updates steps.
        'no_cuda': False,                       # Whether not to use CUDA when available
        'overwrite_cache': True,               # Overwrite the cached training and evaluation sets")
        'save': 'all',                          # Select load mode from ['all', '0', '1', '2', '3', ...]")
    }

    main_args = dotdict(main_args)

    main_QA(main_args)

Epoch: 100%|██████████| 1/1 [00:05<00:00,  5.17s/it]


In [ ]:
#@title Run task embeddings for QA task with DROP

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    def __getstate__(self): return self.__dict__
    def __setstate__(self, d): self.__dict__.update(d)


if __name__ == '__main__':

    main_args = {
        'train_file': 'datasets/drop/train.json',
        'model_type': 'bert',
        'model_name_or_path': 'bert-base-uncased',
        'output_dir': './gdrive/MyDrive/colab-results/drop',
        'dataset_name': 'drop',

        'num_softmax_classifiers': 1,           # Number of softmax classifiers on top of Bert's output.
        'pow': 2.0,                             # Return features to the power pow.
        'feature_type': '',                     # The type of the features selected in ['grads', 'weights']"
        'retain_gradients': True,               # Whether to retain gradients at each layer output of the feature extractor.
        'do_pooling': 'True',                   # Whether to pool the feature extractor.
        'use_labels': 'True',                   # Whether to use training labels or sample from the model's predictive distribution \n" "pθ(y|xn), e.g., to compute the theoretical Fisher information.
        'num_trials_for_FIM': 100,              # Number of trials to sample from the model's predictive distribution pθ(y|xn).
        'FIM_scale': 0.25,                      # Standard deviation of the distribution used to compute the theoretical FIM.
        'finetune_classifier': 'False',         # Whether to fine-tune the final classifier.
        'finetune_feature_extractor': 'False',  # Whether to fine-tune the feature extractor.

        'batch_size': 15,
        'data_subset': 60,
        'overwrite_output_dir': True,
        'device': 'cuda',
        'seed': 42,

        'config_name': '',                      # Pretrained config name or path if not the same as model_name
        'tokenizer_name': '',                   # Pretrained tokenizer name or path if not the same as model_name
        'cache_dir': './cache',                        # Where do you want to store the pre-trained models downloaded from s3
        'version_2_with_negative': False,         # If true, the SQuAD examples contain some that do not have an answer.
        'null_score_diff_threshold': 0.0,       # If null_score - best_non_null is greater than the threshold predict null.
        'max_seq_length': 384,                  # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded.
        'doc_stride': 128,                      # When splitting up a long document into chunks, how much stride to take between chunks.
        'max_query_length': 64,                 # The maximum number of tokens for the question. Questions longer than this will " "be truncated to this length.
        'do_train': True,                       # Whether to run training.
        'do_lower_case': True,                  # Set this flag if you are using an uncased model.
        'learning_rate': 5e-5,                  # The initial learning rate for Adam."
        'gradient_accumulation_steps': 1,       # Number of updates steps to accumulate before performing a backward/update pass.")
        'weight_decay': 0.0,                    # Weight deay if we apply some.
        'adam_epsilon': 1e-8,                   # Epsilon for Adam optimizer.")
        'max_grad_norm': 1.0,                   # Max gradient norm."
        'num_epochs': 1.0,                      # Total number of training epochs to perform.
        'warmup_steps': 0,                      # Linear warmup over warmup_steps.
        'n_best_size': 20,                      # The total number of n-best predictions to generate in the nbest_predictions.json output file.
        'max_answer_length': 30,                # The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        'verbose_logging': True,                # If true, all of the warnings related to data processing will be printed. " "A number of warnings are expected for a normal SQuAD evaluation.
        'logging_steps': 50,                    # Log every X updates steps.
        'no_cuda': False,                       # Whether not to use CUDA when available
        'overwrite_cache': True,               # Overwrite the cached training and evaluation sets")
        'save': 'all',                          # Select load mode from ['all', '0', '1', '2', '3', ...]")
    }

    main_args = dotdict(main_args)

    main_QA(main_args)

Epoch: 100%|██████████| 1/1 [00:05<00:00,  5.19s/it]


In [12]:
#@title Run task embeddings for QA task with duorc_p

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    def __getstate__(self): return self.__dict__
    def __setstate__(self, d): self.__dict__.update(d)


if __name__ == '__main__':

    main_args = {
        'train_file': 'datasets/duorc_p/train.json',
        'model_type': 'bert',
        'model_name_or_path': 'bert-base-uncased',
        'output_dir': './gdrive/MyDrive/colab-results/duorc_p',
        'dataset_name': 'duorc_p',

        'num_softmax_classifiers': 1,           # Number of softmax classifiers on top of Bert's output.
        'pow': 2.0,                             # Return features to the power pow.
        'feature_type': '',                     # The type of the features selected in ['grads', 'weights']"
        'retain_gradients': True,               # Whether to retain gradients at each layer output of the feature extractor.
        'do_pooling': 'True',                   # Whether to pool the feature extractor.
        'use_labels': 'True',                   # Whether to use training labels or sample from the model's predictive distribution \n" "pθ(y|xn), e.g., to compute the theoretical Fisher information.
        'num_trials_for_FIM': 100,              # Number of trials to sample from the model's predictive distribution pθ(y|xn).
        'FIM_scale': 0.25,                      # Standard deviation of the distribution used to compute the theoretical FIM.
        'finetune_classifier': 'False',         # Whether to fine-tune the final classifier.
        'finetune_feature_extractor': 'False',  # Whether to fine-tune the feature extractor.

        'batch_size': 5,
        'data_subset': 20,
        'overwrite_output_dir': True,
        'device': 'cuda',
        'seed': 42,

        'config_name': '',                      # Pretrained config name or path if not the same as model_name
        'tokenizer_name': '',                   # Pretrained tokenizer name or path if not the same as model_name
        'cache_dir': './cache',                 # Where do you want to store the pre-trained models downloaded from s3
        'version_2_with_negative': True,        # If true, the SQuAD examples contain some that do not have an answer.
        'null_score_diff_threshold': 0.0,       # If null_score - best_non_null is greater than the threshold predict null.
        'max_seq_length': 384,                  # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded.
        'doc_stride': 128,                      # When splitting up a long document into chunks, how much stride to take between chunks.
        'max_query_length': 64,                 # The maximum number of tokens for the question. Questions longer than this will " "be truncated to this length.
        'do_train': True,                       # Whether to run training.
        'do_lower_case': True,                  # Set this flag if you are using an uncased model.
        'learning_rate': 5e-5,                  # The initial learning rate for Adam."
        'gradient_accumulation_steps': 1,       # Number of updates steps to accumulate before performing a backward/update pass.")
        'weight_decay': 0.0,                    # Weight deay if we apply some.
        'adam_epsilon': 1e-8,                   # Epsilon for Adam optimizer.")
        'max_grad_norm': 1.0,                   # Max gradient norm."
        'num_epochs': 1.0,                      # Total number of training epochs to perform.
        'warmup_steps': 0,                      # Linear warmup over warmup_steps.
        'n_best_size': 20,                      # The total number of n-best predictions to generate in the nbest_predictions.json output file.
        'max_answer_length': 30,                # The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        'verbose_logging': True,                # If true, all of the warnings related to data processing will be printed. " "A number of warnings are expected for a normal SQuAD evaluation.
        'logging_steps': 50,                    # Log every X updates steps.
        'no_cuda': False,                       # Whether not to use CUDA when available
        'overwrite_cache': True,               # Overwrite the cached training and evaluation sets")
        'save': 'all',                          # Select load mode from ['all', '0', '1', '2', '3', ...]")
    }

    main_args = dotdict(main_args)

    main_QA(main_args)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Iteration:   0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "

Epoch: 100%|██████████| 1/1 [00:05<00:00,  5.87s/it]


In [15]:
#@title Run task embeddings for QA task with duorc_r

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    def __getstate__(self): return self.__dict__
    def __setstate__(self, d): self.__dict__.update(d)


if __name__ == '__main__':

    main_args = {
        'train_file': 'datasets/duorc_r/train.json',
        'model_type': 'bert',
        'model_name_or_path': 'bert-base-uncased',
        'output_dir': './gdrive/MyDrive/colab-results/duorc_r',
        'dataset_name': 'duorc_r',

        'num_softmax_classifiers': 1,           # Number of softmax classifiers on top of Bert's output.
        'pow': 2.0,                             # Return features to the power pow.
        'feature_type': '',                     # The type of the features selected in ['grads', 'weights']"
        'retain_gradients': True,               # Whether to retain gradients at each layer output of the feature extractor.
        'do_pooling': 'True',                   # Whether to pool the feature extractor.
        'use_labels': 'True',                   # Whether to use training labels or sample from the model's predictive distribution \n" "pθ(y|xn), e.g., to compute the theoretical Fisher information.
        'num_trials_for_FIM': 100,              # Number of trials to sample from the model's predictive distribution pθ(y|xn).
        'FIM_scale': 0.25,                      # Standard deviation of the distribution used to compute the theoretical FIM.
        'finetune_classifier': 'False',         # Whether to fine-tune the final classifier.
        'finetune_feature_extractor': 'False',  # Whether to fine-tune the feature extractor.

        'batch_size': 5,
        'data_subset': 20,
        'overwrite_output_dir': True,
        'device': 'cuda',
        'seed': 42,

        'config_name': '',                      # Pretrained config name or path if not the same as model_name
        'tokenizer_name': '',                   # Pretrained tokenizer name or path if not the same as model_name
        'cache_dir': './cache',                        # Where do you want to store the pre-trained models downloaded from s3
        'version_2_with_negative': True,         # If true, the SQuAD examples contain some that do not have an answer.
        'null_score_diff_threshold': 0.0,       # If null_score - best_non_null is greater than the threshold predict null.
        'max_seq_length': 384,                  # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded.
        'doc_stride': 128,                      # When splitting up a long document into chunks, how much stride to take between chunks.
        'max_query_length': 64,                 # The maximum number of tokens for the question. Questions longer than this will " "be truncated to this length.
        'do_train': True,                       # Whether to run training.
        'do_lower_case': True,                  # Set this flag if you are using an uncased model.
        'learning_rate': 5e-5,                  # The initial learning rate for Adam."
        'gradient_accumulation_steps': 1,       # Number of updates steps to accumulate before performing a backward/update pass.")
        'weight_decay': 0.0,                    # Weight deay if we apply some.
        'adam_epsilon': 1e-8,                   # Epsilon for Adam optimizer.")
        'max_grad_norm': 1.0,                   # Max gradient norm."
        'num_epochs': 1.0,                      # Total number of training epochs to perform.
        'warmup_steps': 0,                      # Linear warmup over warmup_steps.
        'n_best_size': 20,                      # The total number of n-best predictions to generate in the nbest_predictions.json output file.
        'max_answer_length': 30,                # The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        'verbose_logging': True,                # If true, all of the warnings related to data processing will be printed. " "A number of warnings are expected for a normal SQuAD evaluation.
        'logging_steps': 50,                    # Log every X updates steps.
        'no_cuda': False,                       # Whether not to use CUDA when available
        'overwrite_cache': True,               # Overwrite the cached training and evaluation sets")
        'save': 'all',                          # Select load mode from ['all', '0', '1', '2', '3', ...]")
    }

    main_args = dotdict(main_args)

    main_QA(main_args)

Epoch: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


In [24]:
#@title Run task embeddings for CR task with SST2

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    def __getstate__(self): return self.__dict__
    def __setstate__(self, d): self.__dict__.update(d)


if __name__ == '__main__':

    main_args = {
        'task_name': 'sst-2',
        'output_dir': './gdrive/MyDrive/colab-results/sst2',
        'model_name_or_path': 'bert-base-uncased',
        'model_type': 'bert',
        'data_dir': 'datasets/sst2',

        'num_softmax_classifiers': 1,           # Number of softmax classifiers on top of Bert's output.
        'pow': 2.0,                             # Return features to the power pow.
        'feature_type': '',                     # The type of the features selected in ['grads', 'weights']"
        'retain_gradients': True,               # Whether to retain gradients at each layer output of the feature extractor.
        'do_pooling': 'True',                   # Whether to pool the feature extractor.
        'use_labels': 'True',                   # Whether to use training labels or sample from the model's predictive distribution \n" "pθ(y|xn), e.g., to compute the theoretical Fisher information.
        'num_trials_for_FIM': 100,              # Number of trials to sample from the model's predictive distribution pθ(y|xn).
        'FIM_scale': 0.25,                      # Standard deviation of the distribution used to compute the theoretical FIM.
        'finetune_classifier': 'False',         # Whether to fine-tune the final classifier.
        'finetune_feature_extractor': 'False',  # Whether to fine-tune the feature extractor.

        'batch_size': 15,
        'data_subset': 60,
        'overwrite_output_dir': True,
        'device': 'cuda',
        'seed': 42,

        'config_name': '',                      # Pretrained config name or path if not the same as model_name
        'tokenizer_name': '',                   # Pretrained tokenizer name or path if not the same as model_name
        'cache_dir': './cache',                        # Where do you want to store the pre-trained models downloaded from s3
        'version_2_with_negative': True,         # If true, the SQuAD examples contain some that do not have an answer.
        'null_score_diff_threshold': 0.0,       # If null_score - best_non_null is greater than the threshold predict null.
        'max_seq_length': 384,                  # The maximum total input sequence length after WordPiece tokenization. Sequences longer than this will be truncated, and sequences shorter than this will be padded.
        'doc_stride': 128,                      # When splitting up a long document into chunks, how much stride to take between chunks.
        'max_query_length': 64,                 # The maximum number of tokens for the question. Questions longer than this will " "be truncated to this length.
        'do_train': True,                       # Whether to run training.
        'do_lower_case': True,                  # Set this flag if you are using an uncased model.
        'learning_rate': 5e-5,                  # The initial learning rate for Adam."
        'gradient_accumulation_steps': 1,       # Number of updates steps to accumulate before performing a backward/update pass.")
        'weight_decay': 0.0,                    # Weight deay if we apply some.
        'adam_epsilon': 1e-8,                   # Epsilon for Adam optimizer.")
        'max_grad_norm': 1.0,                   # Max gradient norm."
        'num_epochs': 1.0,                      # Total number of training epochs to perform.
        'warmup_steps': 0,                      # Linear warmup over warmup_steps.
        'n_best_size': 20,                      # The total number of n-best predictions to generate in the nbest_predictions.json output file.
        'max_answer_length': 30,                # The maximum length of an answer that can be generated. This is needed because the start and end predictions are not conditioned on one another.
        'verbose_logging': True,                # If true, all of the warnings related to data processing will be printed. " "A number of warnings are expected for a normal SQuAD evaluation.
        'logging_steps': 50,                    # Log every X updates steps.
        'no_cuda': False,                       # Whether not to use CUDA when available
        'overwrite_cache': True,               # Overwrite the cached training and evaluation sets")
        'save': 'all',                          # Select load mode from ['all', '0', '1', '2', '3', ...]")
    }

    main_args = dotdict(main_args)

    main_CR(main_args)


Epoch: 100%|██████████| 1/1 [00:02<00:00,  2.50s/it]


In [29]:
#@title Compute diversity

from numpy.linalg import norm

intermediate_tasks = ['squad_v2', 'squad_v1', 'drop', 'duorc_p', 'duorc_r']
target_tasks = ['sst2']

# intermediate_tasks = ['duorc-p', 'duorc-s']
# target_tasks = ['sst2']

for i_task in intermediate_tasks:
  for t_task in target_tasks:
    A = np.load(f'gdrive/MyDrive/colab-results/{i_task}/cls_output.npy')
    B = np.load(f'gdrive/MyDrive/colab-results/{t_task}/cls_output.npy')

    cosine = np.dot(A, B) / (norm(A) * norm(B))

    print(f'Diversity between {i_task} and {t_task} is: {cosine}')



Diversity between squad_v2 and sst2 is: 0.895498514175415
Diversity between squad_v1 and sst2 is: 0.8873575329780579
Diversity between drop and sst2 is: 0.9019232988357544
Diversity between duorc_p and sst2 is: 0.7796794176101685
Diversity between duorc_r and sst2 is: 0.8826728463172913
